In [ ]:
pip install keras-tuner

In [2]:
# importing the libraries

import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.8.2


In [5]:
# importing the dataset

fashion_minst=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_minst.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [12]:
# Resizing

train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [39]:
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('con_2_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [40]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [48]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output1',project_name='Mnist Fashion')

In [49]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.1054999977350235

Best val_accuracy So Far: 0.9076666831970215
Total elapsed time: 00h 03m 13s


In [50]:
model=tuner_search.get_best_models(num_models=1)[0]

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        115248    
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 48)                1115184   
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 1,231,882
Trainable params: 1,231,882
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1594 - accuracy: 0.9417 - val_loss: 0.2517 - val_accuracy: 0.9083
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1257 - accuracy: 0.9531 - val_loss: 0.2632 - val_accuracy: 0.9152
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0943 - accuracy: 0.9650 - val_loss: 0.2897 - val_accuracy: 0.9105
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0741 - accuracy: 0.9728 - val_loss: 0.3506 - val_accuracy: 0.9137
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0561 - accuracy: 0.9789 - val_loss: 0.3696 - val_accuracy: 0.9140
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0442 - accuracy: 0.9839 - val_loss: 0.4569 - val_accuracy: 0.9085
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0376 - accuracy: 0.9867 - val_loss: 0.4469 - val_accuracy